# Build a "no internet" TRex Game

![](images/dino-game.png "Dino")

# Define the game

A 2D, Side-scrolling, infinite, 1 key control video game. It has two scenes, game and game end.

# Game Loop

The game may have multiple round. At the end of game, player can press 'ESC' to quit the game or 'Enter' to do another game.

In [ ]:
if __name__ == '__main__':
    # setup the screen ....
    
    while(True):
        main_game = TRexGame(window)
        main_game.start()
        if not main_game.should_continue():
            break

In [ ]:
    def should_continue(self):
        self.window.clear()
        self.window.border(NO_BORDER)
        self.draw_at(10, 27, [
            "  ___   _   __  __ ___    _____   _____ ___ ",
            " / __| /_\ |  \/  | __|  / _ \ \ / / __| _ \\",
            "| (_ |/ _ \| |\/| | _|  | (_) \ V /| _||   /",
            " \___/_/ \_\_|  |_|___|  \___/ \_/ |___|_|_\\",
            "",
            "             FINAL_SCORE : "+str(self.score),
            "",
            "Press 'Enter' Key to Restart or 'ESC' to Quit"
        ])
        while(True):
            key_event = self.window.getch()
            if key_event is KEY_ESC:
                return False
            elif key_event is KEY_ENTER:
                return True

# Graphic

## FPS - Frame Per Second

FPS means how many images (frames) the computer can render. Higher FPS makes the game feel more real, but it needs more CPU power. 

From the code in the `TRexGame::start` method, can you tell what is our game's FPS?

In [ ]:
    def start(self, window):
        while(not self.is_end_of_game()):
            self.handle_controls()
            self.update_all()
            self.draw_all()
            sleep(0.06)
        sleep(2)
        return self.score

## Sprites / Actors / Game Objects

![](images/sprites.png)

In [ ]:
    def draw_all(self):
        window.clear()
        self.cloud.draw()
        self.ground.draw()
        self.trex.draw()
        self.draw_score()

## Animation

## Background

### Parallax scrolling

![](images/parallax_scroll.gif "Parallel Scrolling")

# Control

# Scoring

# Collision detection 

# Game Plan

## Name:
## Abstract

> .
>
> .


## Game Flow

> .
>
> .

## Graphics & Game Objects

### Sprites / Actors / Game Objects

> .
>
> .

### Background

> .
>
> .

## Scoring

## Game Screens


## Controls

